In [1]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [2]:
df = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/component_training/df_train.csv")

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

max_new_tokens = 512

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
def get_summary(text):
    messages = [
        {"role": "system", "content": "You are a bug report summarizer. " \
        "Given a bug description that includes logs, stacktraces, your job is to summarize the bug in natural language for better understanding."},
        {"role": "user", "content": "Sumamrize the provided bug report in a passage. During summarization avoid using special characters, brackets, etc." 
        "Include every details in the bug presumably the root cause, associated files, any relavant information.:\n" + text},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        max_length=model.config.max_position_embeddings-max_new_tokens,
        truncation=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    
    return tokenizer.decode(response, skip_special_tokens=True)


In [7]:
summaries = []

print("Generating summaries...")
for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing issue"):
    text = f"Bug Title: {row.issue_title}\nBug Description: {row.description}"
    sumamry = get_summary(text)
    sumamry = sumamry.replace("\n\n", "\n")
    summaries.append(sumamry)

Generating summaries...


Processing issue:   0%|          | 0/2472 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing issue:   0%|          | 1/2472 [00:08<5:30:53,  8.03s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing issue:   0%|          | 2/2472 [00:15<5:20:29,  7.79s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Processing issue:   0%|          | 3/2472 [00:20<4

KeyboardInterrupt: 

In [18]:
print(summaries[7])

Here is a summarized version of the bug report in natural language:
A new flag is needed to recognize primitive classes in OpenJ9. According to Java Enhancement Proposal 401, a new modifier called ACC_PRIMITIVE (0x0800) is being added. This modifier should be set for primitive classes, which are also known as value types in OpenJ9 code. Currently, OpenJ9 uses a flag called J9AccValueType (0x100) to recognize value classes, but a new flag is required to recognize primitive classes.


In [20]:
print(df.iloc[7].description)

From https://openjdk.java.net/jeps/401:
A new modifier `ACC_PRIMITIVE` (0x0800) is added.

For primitive class (value type in the current OpenJ9 code), both `ACC_PRIMITIVE` and `ACC_VALUE` are set.
For value class, only `ACC_VALUE` (0x0100) is set.

Currently, OpenJ9 has flag `J9AccValueType` (0x100) for `ACC_VALUE`. New flag needs to be added to recognize `ACC_PRIMITIVE`. 
